##### 라이브러리

###### 라이브러리 설치

In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


###### 라이브러리 불러오기

In [2]:
import pandas as pd

# 기사 데이터 정리 및 통계

### 전체 기사 일자별 카운트

###### 삼성전자 뉴스 csv 불러오기

In [19]:
df_news = pd.read_csv("naver_news/naver_news_20230801_20230831.csv")
df_news.head()

,date,category,title,content,reaction
0,2023-08-01 01:15:00,세계,"니제르 쿠데타 군부 ""프랑스, 군사개입 공모"" 비난(종합)","전임 대통령 아들 포함 각료 4명·여당 대표 체포 중재 나선 차드 수반, '억류중'...","{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
1,2023-08-01 01:20:00,세계,"Trilateral summit of S. Korea, Japan, U.S. to ...",S Korea-US-Japan Trilateral summit of S. Korea...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
2,2023-08-01 01:26:00,정치,"필리핀 간 EU 수장, 中겨냥 작심발언…""불법 무력사용 용인불가""(종합)",신냉전 전 속 美 이어 EU도 필리핀과 관계 개선 모색…FTA 협상 재개키로 정빛...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': 5, '분석탁월': ..."
3,2023-08-01 02:10:00,세계,"머스크 엑스, SNS연구단체에 소송 위협…""광고주 몰아내려 음모""","엑스 ""CCDH, 우리 경쟁사서 자금지원 받아""…CCDH ""비판에 재갈 물리려는 것...","{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
4,2023-08-01 02:11:00,세계,"러 가스차단 겪은 유럽, '우라늄대국' 니제르 쿠데타에 전전긍긍","EU 우라늄 수입 20% 차지…원전 의존도 높은 프랑스도 '벌벌' ""니제르 사태 악...","{'쏠쏠정보': 3, '흥미진진': '0', '공감백배': 3, '분석탁월': 2,..."


###### date 열 데이터 변경

In [20]:
df_news["date"] = df_news["date"].apply(lambda x: x.split()[0].replace("-", ""))
df_news.head()

,date,category,title,content,reaction
0,20230801,세계,"니제르 쿠데타 군부 ""프랑스, 군사개입 공모"" 비난(종합)","전임 대통령 아들 포함 각료 4명·여당 대표 체포 중재 나선 차드 수반, '억류중'...","{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
1,20230801,세계,"Trilateral summit of S. Korea, Japan, U.S. to ...",S Korea-US-Japan Trilateral summit of S. Korea...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
2,20230801,정치,"필리핀 간 EU 수장, 中겨냥 작심발언…""불법 무력사용 용인불가""(종합)",신냉전 전 속 美 이어 EU도 필리핀과 관계 개선 모색…FTA 협상 재개키로 정빛...,"{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': 5, '분석탁월': ..."
3,20230801,세계,"머스크 엑스, SNS연구단체에 소송 위협…""광고주 몰아내려 음모""","엑스 ""CCDH, 우리 경쟁사서 자금지원 받아""…CCDH ""비판에 재갈 물리려는 것...","{'쏠쏠정보': '0', '흥미진진': '0', '공감백배': '0', '분석탁월'..."
4,20230801,세계,"러 가스차단 겪은 유럽, '우라늄대국' 니제르 쿠데타에 전전긍긍","EU 우라늄 수입 20% 차지…원전 의존도 높은 프랑스도 '벌벌' ""니제르 사태 악...","{'쏠쏠정보': 3, '흥미진진': '0', '공감백배': 3, '분석탁월': 2,..."


###### 전체 기사 일자별 카운트

In [21]:
all = []
for day in range(1, 32):
    date_str = f"202308{day:02d}"
    count_all = len(df_news.loc[df_news["date"] == date_str])
    all.append({"날짜":date_str, "전체":count_all})

all

[{'날짜': '20230801', '전체': 939},
 {'날짜': '20230802', '전체': 1012},
 {'날짜': '20230803', '전체': 1119},
 {'날짜': '20230804', '전체': 854},
 {'날짜': '20230805', '전체': 381},
 {'날짜': '20230806', '전체': 552},
 {'날짜': '20230807', '전체': 839},
 {'날짜': '20230808', '전체': 1059},
 {'날짜': '20230809', '전체': 1034},
 {'날짜': '20230810', '전체': 1176},
 {'날짜': '20230811', '전체': 841},
 {'날짜': '20230812', '전체': 251},
 {'날짜': '20230813', '전체': 414},
 {'날짜': '20230814', '전체': 863},
 {'날짜': '20230815', '전체': 526},
 {'날짜': '20230816', '전체': 1029},
 {'날짜': '20230817', '전체': 1044},
 {'날짜': '20230818', '전체': 855},
 {'날짜': '20230819', '전체': 287},
 {'날짜': '20230820', '전체': 428},
 {'날짜': '20230821', '전체': 877},
 {'날짜': '20230822', '전체': 997},
 {'날짜': '20230823', '전체': 1094},
 {'날짜': '20230824', '전체': 1265},
 {'날짜': '20230825', '전체': 838},
 {'날짜': '20230826', '전체': 238},
 {'날짜': '20230827', '전체': 380},
 {'날짜': '20230828', '전체': 943},
 {'날짜': '20230829', '전체': 1101},
 {'날짜': '20230830', '전체': 1086},
 {'날짜': '20230831', '전체': 111

###### 데이터프레임 생성

In [22]:
count_all = pd.DataFrame(all)
count_all.head()

,날짜,전체
0,20230801,939
1,20230802,1012
2,20230803,1119
3,20230804,854
4,20230805,381
